In [1]:
import json
from paperqa import Settings, ask
import os
from paperqa import Settings, Docs
from paperqa.agents.main import agent_query
from paperqa.agents.search import get_directory_index

from aviary.env import TaskDataset
from ldp.agent import SimpleAgent
from ldp.alg.callbacks import MeanMetricsCallback
from ldp.alg.runners import Evaluator, EvaluatorConfig


In [2]:
# Load the questions
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/questions/formatted_questions/questions.json", "r") as f:
    questions = json.load(f)

# Search for this specific question
target_question = "What effect does bone marrow stromal cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?"
matching_questions = [q for q in questions if q["question"] == target_question]

if matching_questions:
    print("Found the question!")
    question_data = matching_questions[0]
else:
    print("Question not found in dataset")

Found the question!


In [3]:
question_data

{'question': 'What effect does bone marrow stromal cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?',
 'choices': ['(A) No effect',
  '(B) Insufficient information to answer the question',
  '(C) Decrease',
  '(D) Increase'],
 'correct_answer': 'A',
 'unsure_option': 'B',
 'sources': ['https://doi.org/10.1101/2024.01.31.578101'],
 'ideal': 'No effect',
 'distractors': ['Increase', 'Decrease']}

In [4]:
def format_multiple_choice_question(question, choices):
    formatted = f"Question:Q: {question}; Options: "
    formatted += " ".join(choices)
    return formatted


In [19]:
# Set up PaperQA2 with the paper directory
settings = Settings(
    temperature=0.0,  # Keep deterministic
    paper_directory="/Users/apple/Documents/GitLab_Projects/master_project/xx823/paper-2a"  # Directory containing the paper with DOI: 10.1101/2024.01.31.578101
)


In [5]:
formatted_q = format_multiple_choice_question(question_data['question'], question_data['choices'])
formatted_q

'Question:Q: What effect does bone marrow stromal cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?; Options: (A) No effect (B) Insufficient information to answer the question (C) Decrease (D) Increase'

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
# Create settings with explicit configuration
settings = Settings(
    paper_directory='/Users/apple/Documents/GitLab_Projects/master_project/xx823/paper-2a',
    agent={"index": {
        "sync_with_paper_directory": True,
        "recurse_subdirectories": True
    }}
)



In [8]:
paper_dir = "/Users/apple/Documents/GitLab_Projects/master_project/xx823/paper-2a"
if not os.path.exists(paper_dir):
    print(f"Directory {paper_dir} does not exist!")
    # Create it
    os.makedirs(paper_dir, exist_ok=True)
else:
    print(f"Directory {paper_dir} exists, checking contents:")
    files = os.listdir(paper_dir)
    print(f"Found {len(files)} files: {files}")

Directory /Users/apple/Documents/GitLab_Projects/master_project/xx823/paper-2a exists, checking contents:
Found 3 files: ['.DS_Store', 'darby-et-al-2024-differential-development-of-antibiotic-resistance-and-virulence-between-acinetobacter-species.pdf', 'The T-cell niche tunes immune function through modulation of the cytoskeleton and TCRantigen forces.pdf']


In [9]:

built_index = await get_directory_index(settings=settings)

# Print index information 
print(f"Using index: {settings.get_index_name()}")
index_files = await built_index.index_files
print(f"Number of indexed files: {len(index_files)}")
print("Indexed files:")
for file in index_files:
    print(f"- {file}")

Using index: pqa_index_f149e57fac17a94e4c4330421b6db4f7
Number of indexed files: 2
Indexed files:
- darby-et-al-2024-differential-development-of-antibiotic-resistance-and-virulence-between-acinetobacter-species.pdf
- The T-cell niche tunes immune function through modulation of the cytoskeleton and TCRantigen forces.pdf


In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from paperqa.settings import Settings, AgentSettings

settings = Settings(
    llm="gpt-4o-mini",
    llm_config={
        "model_list": [
            {
                "model_name": "gpt-4o-mini",
                "litellm_params": {
                    "model": "gpt-4o-mini",
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            "gpt-4o-mini": "30000 per 1 minute",
        },
    },
    summary_llm="gpt-4o-mini",
    summary_llm_config={
        "rate_limit": {
            "gpt-4o-mini": "30000 per 1 minute",
        },
    },
    agent=AgentSettings(
        agent_llm="gpt-4o-mini",
        agent_llm_config={
            "rate_limit": {
                "gpt-4o-mini": "30000 per 1 minute",
            },
        }
    ),
    embedding="text-embedding-3-small",
    temperature=0.5,  # Keep deterministic
    paper_directory="/Users/apple/Documents/GitLab_Projects/master_project/xx823/paper-2a" , # Directory containing the paper with DOI: 10.1101/2024.01.31.578101
    verbosity=3

)

In [12]:
answer_pqa = ask(formatted_q, settings=settings)

PaperQA version: 5.15.0


[15:05:16] Beginning agent 'ToolSelector' run with question 'Question:Q: What effect does bone marrow stromal      
           cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?; Options:  
           (A) No effect (B) Insufficient information to answer the question (C) Decrease (D) Increase' and full   
           settings {'llm': 'gpt-4o-mini', 'llm_config': {'model_list': [{'model_name': 'gpt-4o-mini',             
           'litellm_params': {'model': 'gpt-4o-mini', 'temperature': 0.1, 'max_tokens': 4096}}], 'rate_limit':     
           {'gpt-4o-mini': '30000 per 1 minute'}}, 'summary_llm': 'gpt-4o-mini', 'summary_llm_config':             
           {'rate_limit': {'gpt-4o-mini': '30000 per 1 minute'}}, 'embedding': 'text-embedding-3-small',           
           'embedding_config': None, 'temperature': 0.5, 'batch_size': 1, 'texts_index_mmr_lambda': 1.0,           
           'verbosity': 3, 'answer': {'evidence_k': 10, 'evidence_detailed_citations': True, 'evidence_retrieval': 
           True, 'evidence_summary_length': 'about 100 words', 'evidence_skip_summary': False,                     
           'answer_max_sources': 5, 'max_answer_attempts': None, 'answer_length': 'about 200 words, but can be     
           longer', 'max_concurrent_requests': 4, 'answer_filter_extra_background': False,                         
           'get_evidence_if_no_contexts': True}, 'parsing': {'chunk_size': 5000, 'page_size_limit': 1280000,       
           'use_doc_details': True, 'overlap': 250, 'citation_prompt': 'Provide the citation for the following text
           in MLA Format. Do not write an introductory sentence. If reporting date accessed, the current year is   
           2025\n\n{text}\n\nCitation:', 'structured_citation_prompt': "Extract the title, authors, and doi as a   
           JSON from this MLA citation. If any field can not be found, return it as null. Use title, authors, and  
           doi as keys, author's value should be a list of authors. {citation}\n\nCitation JSON:",                 
           'disable_doc_valid_check': False, 'defer_embedding': False, 'chunking_algorithm':                       
           <ChunkingOptions.SIMPLE_OVERLAP: 'simple_overlap'>, 'doc_filters': None,                                
           'use_human_readable_clinical_trials': False}, 'prompts': {'summary': 'Summarize the excerpt below to    
           help answer a question.\n\nExcerpt from {citation}\n\n----\n\n{text}\n\n----\n\nQuestion:               
           {question}\n\nDo not directly answer the question, instead summarize to give evidence to help answer the
           question. Stay detailed; report specific numbers, equations, or direct quotes (marked with quotation    
           marks). Reply "Not applicable" if the excerpt is irrelevant. At the end of your response, provide an    
           integer score from 1-10 on a newline indicating relevance to question. Do not explain your              
           score.\n\nRelevant Information Summary ({summary_length}):', 'qa': 'Answer the question below with the  
           context.\n\nContext (with relevance scores):\n\n{context}\n\n----\n\nQuestion: {question}\n\nWrite an   
           answer based on the context. If the context provides insufficient information reply "I cannot answer."  
           For each part of your answer, indicate which sources most support it via citation keys at the end of    
           sentences, like {example_citation}. Only cite from the context below and only use the valid keys. Write 
           in the style of a Wikipedia article, with concise sentences and coherent paragraphs. The context comes  
           from a variety of sources and is only a summary, so there may inaccuracies or ambiguities. If quotes are
           present and relevant, use them in the answer. This answer will go directly onto Wikipedia, so do not add
           any extraneous information.\n\nAnswer ({answe

           No changes to index.

[15:05:18] Starting paper search for 'bone marrow stromal cell-conditioned media CD8a receptor OT-1 T cells'.

           paper_search for query 'bone marrow stromal cell-conditioned media CD8a receptor OT-1 T cells' and      
           offset 0 returned 2 papers.

...

[15:05:33] Status: Paper Count=2 | Relevant Papers=1 | Current Evidence=4 | Current Cost=$0.0042

[15:05:34] Completing 'Question:Q: What effect does bone marrow stromal cell-conditioned media have on the         
           expression of the CD8a receptor in cultured OT-1 T cells?; Options: (A) No effect (B) Insufficient      
           information to answer the question (C) Decrease (D) Increase' as 'certain'.

           Finished agent 'ToolSelector' run with question 'Question:Q: What effect does bone marrow stromal       
           cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?; Options:  
           (A) No effect (B) Insufficient information to answer the question (C) Decrease (D) Increase' and status 
           success.

           agent_response: session=PQASession(id=UUID('4298e26f-7742-456b-bf7b-5fe837644897'),                     
           question='Question:Q: What effect does bone marrow stromal cell-conditioned media have on the expression
           of the CD8a receptor in cultured OT-1 T cells?; Options: (A) No effect (B) Insufficient information to  
           answer the question (C) Decrease (D) Increase', answer='Bone marrow stromal cell-conditioned media (SCM)
           does not significantly affect the expression of the CD8α receptor in cultured OT-1 T cells. Statistical 
           analysis indicated no significant differences in CD8α expression among T-cells treated with             
           unconditioned medium (UCM), SCM, or adipocyte-conditioned medium (ACM), with P values greater than 0.7  
           for comparisons between these conditions (kellner2024thetcellniche pages 3-5). Specifically, when OT-1  
           T-cells were cultured in SCM for 24 hours, there were no significant alterations in the surface         
           expression of the T-cell receptor (TCR) or the CD8 co-receptor (kellner2024thetcellniche pages 5-6).    
           \n\nWhile there was a noted trend towards increased surface expression of CD8α in T-cells cultured in   
           ACM, this suggests that SCM does not enhance CD8α expression compared to UCM (kellner2024thetcellniche  
           pages 5-6). Therefore, the most accurate conclusion regarding the effect of SCM on CD8α expression in   
           OT-1 T cells is that it has no effect. \n\nBased on the provided context, the correct answer to the     
           question is (A) No effect.', has_successful_answer=True, context='kellner2024thetcellniche pages 3-5:   
           The study investigates the effects of different microenvironments on OT-1 T-cell activation,            
           specifically comparing unconditioned medium (UCM), bone marrow stromal cell-conditioned media (SCM), and
           adipocyte-conditioned medium (ACM). The results indicate that there are no significant differences in   
           CD8α expression among the T-cells treated with UCM, SCM, or ACM, as shown by the statistical analysis (P
           values for UCM vs. SCM, UCM vs. ACM, and SCM vs. ACM all being greater than 0.7). This suggests that the
           bone marrow stromal cell-conditioned media does not significantly affect CD8α receptor expression in    
           OT-1 T cells.\nFrom Anna V. Kellner, Rae Hunter, Priscilla Do, Joel Eggert, Maya Jaffe, Delaney K.      
           Geitgey, Miyoung Lee, Jamie A. G. Hamilton, Anthony J. Ross, Raira S. Ank, Rachel L. Bender, Rong Ma,   
           Christopher C. Porter, Erik C. Dreaden, Byron B. Au-Yeung, Karmella A. Haynes, Curtis J. Henry, and     
           Khalid Salaita. The t-cell niche tunes immune function through modulation of the cytoskeleton and       
           tcr-antigen forces. BioRxiv, Feb 2024. URL: https://doi.org/10.1101/2024.01.31.578101,                  
           doi:10.1101/2024.01.31.578101. This article has 2 citations.\n\nkellner2024thetcellniche pages 5-6: The 
           excerpt indicates that when OT-1 T-cells were cultured in bone marrow stromal cell-conditioned medium   
           (SCM) for 24 hours, there were no significant alterations in the surface expression of the T-cell       
           receptor (TCR) or the CD8 co-receptor. However, there was a noted trend towards increased surface       
           expression of CD8α in T-cells cultured in adipocyte-conditioned medium (ACM), suggesting that while SCM 
           does not significantly affect CD8α expression, ACM may have a different impact. Overall, the data       
           suggest that SCM does not enhance CD8α expression in OT-1 T-cells compared to unconditioned medium      
           (UCM).\nFrom Anna V. Kellner, Rae Hunter, Priscilla Do, Joel Eggert, Maya Jaffe, Delaney K. Geitgey,    
           Miyoung Lee, Jamie A. G. Hamilton, Anthony J.

[15:05:35] Answer: Bone marrow stromal cell-conditioned media (SCM) does not significantly affect the expression of
           the CD8α receptor in cultured OT-1 T cells. Statistical analysis indicated no significant differences in
           CD8α expression among T-cells treated with unconditioned medium (UCM), SCM, or adipocyte-conditioned    
           medium (ACM), with P values greater than 0.7 for comparisons between these conditions                   
           (kellner2024thetcellniche pages 3-5). Specifically, when OT-1 T-cells were cultured in SCM for 24 hours,
           there were no significant alterations in the surface expression of the T-cell receptor (TCR) or the CD8 
           co-receptor (kellner2024thetcellniche pages 5-6).                                                       
                                                                                                                   
           While there was a noted trend towards increased surface expression of CD8α in T-cells cultured in ACM,  
           this suggests that SCM does not enhance CD8α expression compared to UCM (kellner2024thetcellniche pages 
           5-6). Therefore, the most accurate conclusion regarding the effect of SCM on CD8α expression in OT-1 T  
           cells is that it has no effect.                                                                         
                                                                                                                   
           Based on the provided context, the correct answer to the question is (A) No effect.

In [43]:
grader_input_data = []

In [44]:
print("Answer:", answer_pqa)

Answer: session=PQASession(id=UUID('be3029a0-943f-469b-bbb6-35061d52f049'), question='Question:Q: What effect does bone marrow stromal cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?; Options: (A) No effect (B) Insufficient information to answer the question (C) Decrease (D) Increase', answer='Bone marrow stromal cell-conditioned media (SCM) does not significantly alter the expression of the CD8α receptor in cultured OT-1 T cells. Studies indicate that there is no significant difference in CD8α expression between OT-1 T cells treated with SCM and those treated with unconditioned medium (UCM) or adipocyte-conditioned medium (ACM). Statistical analysis shows that the p-values for these comparisons are all above 0.05, suggesting a lack of significant effect (kellner2024thetcellniche pages 3-5).\n\nFurthermore, while SCM does not negatively impact CD8α expression, it also does not enhance it significantly. In contrast, a trend towards increased 

In [45]:
# Assuming `response` is the object you want to inspect
attributes = [attr for attr in dir(answer_pqa) if not attr.startswith('_')]
print(attributes)

['bibtex', 'construct', 'copy', 'dict', 'duration', 'from_orm', 'get_summary', 'json', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'session', 'stats', 'status', 'strip_answer', 'timing_info', 'update_forward_refs', 'validate']


In [46]:
answer_dict=answer_pqa.dict()

/var/folders/wg/h6z2ybxs0bnfsz9zc363lx_w0000gn/T/ipykernel_27485/2588850827.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  answer_dict=answer_pqa.dict()


In [47]:
print(answer_dict.keys())

dict_keys(['session', 'bibtex', 'status', 'timing_info', 'duration', 'stats'])


In [48]:
print(answer_dict['session']['answer'])

Bone marrow stromal cell-conditioned media (SCM) does not significantly alter the expression of the CD8α receptor in cultured OT-1 T cells. Studies indicate that there is no significant difference in CD8α expression between OT-1 T cells treated with SCM and those treated with unconditioned medium (UCM) or adipocyte-conditioned medium (ACM). Statistical analysis shows that the p-values for these comparisons are all above 0.05, suggesting a lack of significant effect (kellner2024thetcellniche pages 3-5).

Furthermore, while SCM does not negatively impact CD8α expression, it also does not enhance it significantly. In contrast, a trend towards increased CD8α expression was observed in T cells cultured in ACM, indicating that SCM maintains CD8α levels without promoting an increase (kellner2024thetcellniche pages 5-6). Overall, the evidence suggests that SCM has no effect on CD8α receptor expression in OT-1 T cells, supporting the conclusion that the correct answer is (A) No effect (kellner2

In [49]:
grader_input_data.append({
        "question": question_data["question"],
        "choices": question_data["choices"],
        "response": answer_dict['session']['answer'],
        "correct_answer": question_data["correct_answer"],
        "unsure_option": question_data["unsure_option"]
    })

In [50]:
import sys

In [51]:
sys.path.append("src")

In [52]:
from MC_Grader import MultipleChoiceGrader, process_results

In [53]:
from config import OPENAI_API_KEY
from openai import OpenAI

In [54]:
grader = MultipleChoiceGrader(
    # Add your OpenAI API key here if not set in environment
    openai_api_key=OPENAI_API_KEY,
    results_dir="grading_results"
)


In [55]:
# Use the fixed version instead of the original
metrics = process_results(grader, grader_input_data, save_prefix="paperqa2_evaluation")

2025-03-02 12:34:00,079 - MC_Grader - INFO - Processing question 1/1
2025-03-02 12:34:00,081 - MC_Grader - INFO - Processing question: What effect does bone marrow stromal cell-conditio...


HumanProxy (to GraderAgent):


    Question:
    What effect does bone marrow stromal cell-conditioned media have on the expression of the CD8a receptor in cultured OT-1 T cells?

    Options:
    (A) No effect (B) Insufficient information to answer the question (C) Decrease (D) Increase

    PaperQA2 Response:
    Bone marrow stromal cell-conditioned media (SCM) does not significantly alter the expression of the CD8α receptor in cultured OT-1 T cells. Studies indicate that there is no significant difference in CD8α expression between OT-1 T cells treated with SCM and those treated with unconditioned medium (UCM) or adipocyte-conditioned medium (ACM). Statistical analysis shows that the p-values for these comparisons are all above 0.05, suggesting a lack of significant effect (kellner2024thetcellniche pages 3-5).

Furthermore, while SCM does not negatively impact CD8α expression, it also does not enhance it significantly. In contrast, a trend towards increased CD8α expression was obser

2025-03-02 12:34:01,926 - MC_Grader - INFO - Results saved to grading_results



Results saved to grading_results
- CSV: paperqa2_evaluation_extracted_answers.csv
- JSON: paperqa2_evaluation_extraction_results.json

You can now manually verify the results and calculate metrics.
